# Extra-Pによって出力されたモデルと卒研の4モデルを比較するためのノート

* 問題サイズに対する予測：データ数 4 の平均絶対誤差率よりデータ数 3 の平均絶対誤差率が低い関数
* コア数に対する予測：平均絶対誤差率が0.1%より大きい関数

のプロット・図示を行う。

また、このノートでは最初のポイントが障害となっていると思われるものについては、最初のポイントを除外した上でモデルを生成し、プロットする。

In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb
InteractiveShell.ast_node_interactivity = 'last_expr'

In [ ]:
FixedClassDict = {}
for benchmark in benchmarks:
    targetDF = return_ErrorRateFixedClassDF(benchmark=benchmark, FixClass="B", predict_process=256)
    columns = targetDF.columns.tolist()
    # 平均絶対誤差率が0.1より大きいものを抽出
    targetDF = targetDF[targetDF[columns[0]] > 0.1]
    # データが同じことに由来する重複を削除
    targetDF = targetDF[~targetDF.duplicated()]
    targetFunctionNameList = targetDF.index.tolist()
    FixedClassDict[benchmark] = targetFunctionNameList
FixedClassDict

# Extra-Pへの入力ファイルを生成
funcCallNumDict = {}
for benchmarkName in FixedClassDict:
    print(f"-----{benchmarkName}-----")
    if benchmarkName == "bt" or benchmarkName == "sp" :
        _processes = process_onlyBTSP
    else:
        _processes = process_excludeBTSP
    funcCallNumDict[benchmarkName] = {}
    for functionName in FixedClassDict[benchmarkName]:
        funcCallNumDict[benchmarkName][functionName] = returnRawDF(Benchmark = benchmarkName, functionName = functionName, fix="Class", Processes = _processes, FixedBenchmarkClass="B")
#         generateInputFileForExtraP(funcCallNumDict[benchmarkName][functionName], repeated=3, includeLastData = False, fileName = f"tmp_{benchmarkName}_{functionName}.txt")
        print(f"_____\"{functionName}\"_____")

> * コア数に対する予測：平均絶対誤差率が0.1%より大きい関数

In [ ]:
plt.figure(figsize=(10,10),dpi=200)


# ベンチマーク名と関数名の指定
benchmarkName = "ft"
functionName = "CFFTZ"
print(f"___________________\"{benchmarkName}\"_____\"{functionName}\"___________________")

# 実データの取り出し
functionDF = funcCallNumDict[benchmarkName][functionName]
functionDF
raw_x = functionDF.columns.tolist()
raw_y = [functionDF.at[functionName, x] for x in raw_x]

# Extra-P用の入力データの作成
generateInputFileForExtraPfromLists(input_x=raw_x, input_y=raw_y, repeated=3, includeFirstData=False, includeLastData=False, fileName =f"tmp_{benchmarkName}_{functionName}.txt")
raw_x = np.array(raw_x).reshape(-1,1)
raw_y = np.array(raw_y).reshape(-1, 1)
# Extra-PでfixProcessデータを入力して出力したモデルの図示
plot_x = np.linspace(0.01, 256, 500)
# 下記はExtra-Pで出力されたモデル
# 56136.0
plot_y = []
plot_y = [56136.0 for x in plot_x]
plt.plot(plot_x, plot_y, label="ExtraP")

# 説明変数と目的変数に分割
train_x, target_x = raw_x[:-1], raw_x[-1]
train_y, target_y = raw_y[:-1], raw_y[-1]

plt.scatter(train_x, train_y, marker="o", label="予測に用いた関数コール回数")
plt.scatter(target_x, target_y, marker="o", label="予測したい関数コール回数の実測値")

# モデル式をプロットするために変数”plot_x”に対して変換を実施
plot_x = np.array(plot_x).reshape(-1, 1)

# 線形モデル
model_lin = ModelLin(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_lin.calc_lr()
plot_y_lin = model_lin.predict(plot_x)
plt.plot(plot_x, plot_y_lin, label="線形モデル")
# 対数モデル
model_log10 = ModelLog10(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_log10.calc_lr()
plot_y_log10 = model_log10.predict(plot_x)
plt.plot(plot_x, plot_y_log10, label="対数モデル")
# 反比例モデル
model_ip = ModelIP(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_ip.calc_lr()
plot_y_ip = model_ip.predict(plot_x)
plt.plot(plot_x, plot_y_ip, label="反比例モデル")
# 線形飽和モデル
model_branch = ModelBranch(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_branch.calc_lr()
plot_y_branch = model_branch.predict(plot_x)
plt.plot(plot_x, plot_y_branch, label="線形飽和モデル")
# 凡例の表示
plt.legend()
# 軸ラベルの設定
plt.ylabel("関数コール回数")
plt.xlabel("実行コア数")
# 軸の対数軸化
# plt.xscale('log')
plt.yscale('log')

In [ ]:
plt.figure(figsize=(10,10),dpi=200)


# ベンチマーク名と関数名の指定
benchmarkName = "ft"
functionName = "FFTZ2"
print(f"___________________\"{benchmarkName}\"_____\"{functionName}\"___________________")

# 実データの取り出し
functionDF = funcCallNumDict[benchmarkName][functionName]
functionDF
raw_x = functionDF.columns.tolist()
raw_y = [functionDF.at[functionName, x] for x in raw_x]

# Extra-P用の入力データの作成
generateInputFileForExtraPfromLists(input_x=raw_x, input_y=raw_y, repeated=3, includeFirstData=False, includeLastData=False, fileName =f"tmp_{benchmarkName}_{functionName}.txt")
raw_x = np.array(raw_x).reshape(-1,1)
raw_y = np.array(raw_y).reshape(-1, 1)
# Extra-PでfixProcessデータを入力して出力したモデルの図示
plot_x = np.linspace(0.01, 256, 500)
# 下記はExtra-Pで出力されたモデル
# 56136.0
plot_y = []
plot_y = [56136.0 for x in plot_x]
plt.plot(plot_x, plot_y, label="ExtraP")

# 説明変数と目的変数に分割
train_x, target_x = raw_x[:-1], raw_x[-1]
train_y, target_y = raw_y[:-1], raw_y[-1]

plt.scatter(train_x, train_y, marker="o", label="予測に用いた関数コール回数")
plt.scatter(target_x, target_y, marker="o", label="予測したい関数コール回数の実測値")

# モデル式をプロットするために変数”plot_x”に対して変換を実施
plot_x = np.array(plot_x).reshape(-1, 1)

# 線形モデル
model_lin = ModelLin(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_lin.calc_lr()
plot_y_lin = model_lin.predict(plot_x)
plt.plot(plot_x, plot_y_lin, label="線形モデル")
# 対数モデル
model_log10 = ModelLog10(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_log10.calc_lr()
plot_y_log10 = model_log10.predict(plot_x)
plt.plot(plot_x, plot_y_log10, label="対数モデル")
# 反比例モデル
model_ip = ModelIP(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_ip.calc_lr()
plot_y_ip = model_ip.predict(plot_x)
plt.plot(plot_x, plot_y_ip, label="反比例モデル")
# 線形飽和モデル
model_branch = ModelBranch(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_branch.calc_lr()
plot_y_branch = model_branch.predict(plot_x)
plt.plot(plot_x, plot_y_branch, label="線形飽和モデル")
# 凡例の表示
plt.legend()
# 軸ラベルの設定
plt.ylabel("関数コール回数")
plt.xlabel("実行コア数")
# 軸の対数軸化
# plt.xscale('log')
plt.yscale('log')

In [ ]:
plt.figure(figsize=(10,10),dpi=200)


# ベンチマーク名と関数名の指定
benchmarkName = "ft"
functionName = "SET_CLASS"
print(f"___________________\"{benchmarkName}\"_____\"{functionName}\"___________________")

# 実データの取り出し
functionDF = funcCallNumDict[benchmarkName][functionName]
functionDF
raw_x = functionDF.columns.tolist()
raw_y = [functionDF.at[functionName, x] for x in raw_x]

# Extra-P用の入力データの作成
generateInputFileForExtraPfromLists(input_x=raw_x, input_y=raw_y, repeated=3, includeFirstData=False, includeLastData=False, fileName =f"tmp_{benchmarkName}_{functionName}.txt")
raw_x = np.array(raw_x).reshape(-1,1)
raw_y = np.array(raw_y).reshape(-1, 1)
# Extra-PでfixProcessデータを入力して出力したモデルの図示
plot_x = np.linspace(0.01, 256, 500)
# 下記はExtra-Pで出力されたモデル
# 56136.0
plot_y = []
plot_y = [56136.0 for x in plot_x]
plt.plot(plot_x, plot_y, label="ExtraP")

# 説明変数と目的変数に分割
train_x, target_x = raw_x[:-1], raw_x[-1]
train_y, target_y = raw_y[:-1], raw_y[-1]

plt.scatter(train_x, train_y, marker="o", label="予測に用いた関数コール回数")
plt.scatter(target_x, target_y, marker="o", label="予測したい関数コール回数の実測値")

# モデル式をプロットするために変数”plot_x”に対して変換を実施
plot_x = np.array(plot_x).reshape(-1, 1)

# 線形モデル
model_lin = ModelLin(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_lin.calc_lr()
plot_y_lin = model_lin.predict(plot_x)
plt.plot(plot_x, plot_y_lin, label="線形モデル")
# 対数モデル
model_log10 = ModelLog10(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_log10.calc_lr()
plot_y_log10 = model_log10.predict(plot_x)
plt.plot(plot_x, plot_y_log10, label="対数モデル")
# 反比例モデル
model_ip = ModelIP(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_ip.calc_lr()
plot_y_ip = model_ip.predict(plot_x)
plt.plot(plot_x, plot_y_ip, label="反比例モデル")
# 線形飽和モデル
model_branch = ModelBranch(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_branch.calc_lr()
plot_y_branch = model_branch.predict(plot_x)
plt.plot(plot_x, plot_y_branch, label="線形飽和モデル")
# 凡例の表示
plt.legend()
# 軸ラベルの設定
plt.ylabel("関数コール回数")
plt.xlabel("実行コア数")
# 軸の対数軸化
# plt.xscale('log')
plt.yscale('log')

In [ ]:
plt.figure(figsize=(10,10),dpi=200)


# ベンチマーク名と関数名の指定
benchmarkName = "is"
functionName = "MPI_Irecv()"
print(f"___________________\"{benchmarkName}\"_____\"{functionName}\"___________________")

# 実データの取り出し
functionDF = funcCallNumDict[benchmarkName][functionName]
functionDF
raw_x = functionDF.columns.tolist()
raw_y = [functionDF.at[functionName, x] for x in raw_x]

# Extra-P用の入力データの作成
generateInputFileForExtraPfromLists(input_x=raw_x, input_y=raw_y, repeated=3, includeFirstData=False, includeLastData=False, fileName =f"tmp_{benchmarkName}_{functionName}.txt")
raw_x = np.array(raw_x).reshape(-1,1)
raw_y = np.array(raw_y).reshape(-1, 1)
# Extra-PでfixProcessデータを入力して出力したモデルの図示
plot_x = np.linspace(0.01, 256, 500)
# 下記はExtra-Pで出力されたモデル
# 56136.0
plot_y = []
plot_y = [56136.0 for x in plot_x]
plt.plot(plot_x, plot_y, label="ExtraP")

# 説明変数と目的変数に分割
train_x, target_x = raw_x[:-1], raw_x[-1]
train_y, target_y = raw_y[:-1], raw_y[-1]

plt.scatter(train_x, train_y, marker="o", label="予測に用いた関数コール回数")
plt.scatter(target_x, target_y, marker="o", label="予測したい関数コール回数の実測値")

# モデル式をプロットするために変数”plot_x”に対して変換を実施
plot_x = np.array(plot_x).reshape(-1, 1)

# 線形モデル
model_lin = ModelLin(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_lin.calc_lr()
plot_y_lin = model_lin.predict(plot_x)
plt.plot(plot_x, plot_y_lin, label="線形モデル")
# 対数モデル
model_log10 = ModelLog10(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_log10.calc_lr()
plot_y_log10 = model_log10.predict(plot_x)
plt.plot(plot_x, plot_y_log10, label="対数モデル")
# 反比例モデル
model_ip = ModelIP(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_ip.calc_lr()
plot_y_ip = model_ip.predict(plot_x)
plt.plot(plot_x, plot_y_ip, label="反比例モデル")
# 線形飽和モデル
model_branch = ModelBranch(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_branch.calc_lr()
plot_y_branch = model_branch.predict(plot_x)
plt.plot(plot_x, plot_y_branch, label="線形飽和モデル")
# 凡例の表示
plt.legend()
# 軸ラベルの設定
plt.ylabel("関数コール回数")
plt.xlabel("実行コア数")
# 軸の対数軸化
# plt.xscale('log')
plt.yscale('log')

In [ ]:
plt.figure(figsize=(10,10),dpi=200)


# ベンチマーク名と関数名の指定
benchmarkName = "is"
functionName = "double_randlc(double_*_double_*)"
print(f"___________________\"{benchmarkName}\"_____\"{functionName}\"___________________")

# 実データの取り出し
functionDF = funcCallNumDict[benchmarkName][functionName]
functionDF
raw_x = functionDF.columns.tolist()
raw_y = [functionDF.at[functionName, x] for x in raw_x]

# Extra-P用の入力データの作成
generateInputFileForExtraPfromLists(input_x=raw_x, input_y=raw_y, repeated=3, includeFirstData=False, includeLastData=False, fileName =f"tmp_{benchmarkName}_{functionName}.txt")
raw_x = np.array(raw_x).reshape(-1,1)
raw_y = np.array(raw_y).reshape(-1, 1)
# Extra-PでfixProcessデータを入力して出力したモデルの図示
plot_x = np.linspace(0.01, 256, 500)
# 下記はExtra-Pで出力されたモデル
# 56136.0
plot_y = []
plot_y = [56136.0 for x in plot_x]
plt.plot(plot_x, plot_y, label="ExtraP")

# 説明変数と目的変数に分割
train_x, target_x = raw_x[:-1], raw_x[-1]
train_y, target_y = raw_y[:-1], raw_y[-1]

plt.scatter(train_x, train_y, marker="o", label="予測に用いた関数コール回数")
plt.scatter(target_x, target_y, marker="o", label="予測したい関数コール回数の実測値")

# モデル式をプロットするために変数”plot_x”に対して変換を実施
plot_x = np.array(plot_x).reshape(-1, 1)

# 線形モデル
model_lin = ModelLin(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_lin.calc_lr()
plot_y_lin = model_lin.predict(plot_x)
plt.plot(plot_x, plot_y_lin, label="線形モデル")
# 対数モデル
model_log10 = ModelLog10(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_log10.calc_lr()
plot_y_log10 = model_log10.predict(plot_x)
plt.plot(plot_x, plot_y_log10, label="対数モデル")
# 反比例モデル
model_ip = ModelIP(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_ip.calc_lr()
plot_y_ip = model_ip.predict(plot_x)
plt.plot(plot_x, plot_y_ip, label="反比例モデル")
# 線形飽和モデル
model_branch = ModelBranch(train_x, train_y, benchmarkName, functionName, test_ratio=0)
model_branch.calc_lr()
plot_y_branch = model_branch.predict(plot_x)
plt.plot(plot_x, plot_y_branch, label="線形飽和モデル")
# 凡例の表示
plt.legend()
# 軸ラベルの設定
plt.ylabel("関数コール回数")
plt.xlabel("実行コア数")
# 軸の対数軸化
# plt.xscale('log')
plt.yscale('log')